In [ ]:
import os
import time

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle

import solution_search
from ac4 import AC4
from costCalcuation.distributions.create_distribtion_helper import create_helper_for_distribution
from depth_first_search_solver import DepthFirstSearchSolver
from experiment_details import ExperimentDetails
from parse_input import parse_xml, parse_itc2007_curriculum_based, parse_itc2007_post_enrolment
from solution_search import SolutionSearch
from costCalcuation.distributions.create_distribtion_helper import create_helper_for_distribution




In [ ]:
itc2007_track2_path = 'D:\\Desktop\\Datasets\\post'
itc2007_track3_path = 'D:\\Desktop\\Datasets\\curriculum'
itc2019_path = 'D:\\Desktop\\Datasets\\2019'

In [ ]:
def get_all_files(path):
    return [os.path.join(dirpath, file) for dirpath, _, filenames in os.walk(path) for file in filenames]    

In [ ]:
itc2007_track2_files = get_all_files(itc2007_track2_path)
itc2007_track3_files = get_all_files(itc2007_track3_path)
itc2019_files = get_all_files(itc2019_path)

# Get statistics

## Statistics for ITC2019

In [ ]:
problems_and_stats = [parse_xml(file) for file in itc2019_files]
stats = [problem[1] for problem in problems_and_stats]

df = pd.DataFrame(stats)

df.index += 1
df.reset_index(inplace=True,names='index')

os.makedirs('output/stats', exist_ok=True)
df.to_csv('output/stats/itc2019_stats.csv',index=False)


## Statistics for ITC2007 Track 2

In [ ]:
problems_and_stats = [parse_itc2007_post_enrolment(file) for file in itc2007_track2_files]
stats = [problem[1] for problem in problems_and_stats]

df = pd.DataFrame(stats)


#make index 1 based and name them as their file names ex. 1,2,...
df.index += 1
df.reset_index(inplace=True,names='name')

df.index += 1
df.reset_index(inplace=True,names='index')

os.makedirs('output/stats', exist_ok=True)
df.to_csv('output/stats/itc2007_track2_stats.csv',index=False)

## Statistics for ITC2007 Track 3

In [ ]:
problems_and_stats = [parse_itc2007_curriculum_based(file) for file in itc2007_track3_files]
stats = [problem[1] for problem in problems_and_stats]

df = pd.DataFrame(stats)


df.index += 1
df.reset_index(inplace=True,names='index')

os.makedirs('output/stats', exist_ok=True)
df.to_csv('output/stats/itc2007_track3_stats.csv',index=False)


# Construction using depth first search

In [ ]:
def plot_row_over_operation(file,operation_history , title):
    fig,ax = plt.subplots()
    ax.set_title(title)
    
    y = [i['current_row'] for i in operation_history]
    
    ax.set_xlabel('Operation')
    ax.set_ylabel('Classes placed')
    
    ax.plot(y)
    plt.show()
    
    if file:
        fig.savefig(file + '.png')
    
    
    
def plot_row_over_time(file, operation_history , title):
    fig,ax = plt.subplots()
    ax.set_title(title)
    
    time = [i['time'] for i in operation_history]
    row = [i['current_row'] for i in operation_history]
    
    ax.set_xlabel('Time')
    ax.set_ylabel('Classes placed')
    ax.plot(time,row)

    plt.show()
    
    if file:
        fig.savefig(file + '.png')
    

In [ ]:
def parse_problem (dataset,instance_path):
    if dataset == 'itc2007_track2':
        return parse_itc2007_post_enrolment(instance_path)
    elif dataset == 'itc2007_track3':
        return parse_itc2007_curriculum_based(instance_path)
    elif dataset == 'itc2019':
        return parse_xml(instance_path)


def run_experiment(experiment_name, problem_files, dataset, ac4_params=None, depth_first_params=None):
    
    folder = f'output/{experiment_name}_{dataset}_{time.strftime("%Y%m%d-%H%M%S")}'
    os.mkdir(folder)
    
    print(f'experiment results will be saved in {folder}')
    
    for f in problem_files:
        date = time.time()
        problem = parse_problem(dataset,f)[0]        
        sub_folder = os.path.join(folder, f.split('\\')[-1].split('.')[0])
        os.mkdir(sub_folder)
        
        res = {}
        
        search = None
        if ac4_params or depth_first_params:
            solution_search_init_start_time = time.time()
            search = SolutionSearch(problem)
            solution_search_init_end_time = time.time()
            res['solution_search_init_time'] = solution_search_init_end_time - solution_search_init_start_time            
            
        
        if ac4_params:
            
            ac4_res = {}
            
            ac4_start_time = time.time()
            ac4 = AC4(search)            
            ac4_res['ac4_init_time'] = time.time() - ac4_start_time
            
            ac4_apply_start_time = time.time()
            ac4_return_value = ac4.apply(**ac4_params)
            ac4_end_time = time.time()
            ac4_res['ac4_apply_time'] = ac4_end_time - ac4_apply_start_time
            
            ac4_res['ac4_total_time'] = ac4_end_time - ac4_start_time
            
            options_open_before_ac4 = np.count_nonzero(search.decision_table == 0)
            options_open_after_ac4 = np.count_nonzero(search.decision_table == 0)
            options_closed_by_ac4 = options_open_before_ac4 - options_open_after_ac4
            
            ac4_res['options_open_before_ac4'] = options_open_before_ac4
            ac4_res['options_open_after_ac4'] = options_open_after_ac4
            ac4_res['options_closed_by_ac4'] = options_closed_by_ac4
            ac4_res['ac4_return_value'] = ac4_return_value
            
            if ac4_return_value:
                pickle.dump(search.decision_table, open(os.path.join(sub_folder, 'decision_table_after_ac4.pkl'), 'wb'))
                
            res['ac4_results'] = ac4_res
            
        if depth_first_params:
            solver = DepthFirstSearchSolver(search)            
            depth_first_res = solver.solve(**depth_first_params)
            
            if depth_first_res['success']:
                np.save(os.path.join(sub_folder, 'solution gene after depth first search.npy'), search.get_result_as_gene())
            
            
            instance_file_name = f.split('\\')[-1].split('.')[0]
            instance_name = problem.name if dataset == 'itc2019' else instance_file_name
            
            plot_row_over_operation(os.path.join(sub_folder, 'row_over_operation_graph.png'), depth_first_res['operation_history'], f"{experiment_name} - {dataset} - {instance_name}")
            plot_row_over_time(os.path.join(sub_folder, 'row_over_time_graph.png'),depth_first_res['operation_history'], f"{experiment_name} - {dataset} - {instance_name}")
            
            res['depth_first_search_results'] = depth_first_res
        
        
        experiment_details = ExperimentDetails(
            name=experiment_name,
            instance_name=instance_name,
            instance_file_name = instance_file_name,
            dataset_name=dataset,
            date=date,
            results_data=res,
            experiment_data={
                'ac4_params': ac4_params,
                'depth_first_params': depth_first_params
            }
        )
        
        pickle.dump(experiment_details, open(os.path.join(sub_folder, 'experiment_details.pkl'), 'wb'))
        

In [ ]:
# folder = 'output/itc2007_track2_depth_first_search_test_'+time.strftime("%Y%m%d-%H%M%S")
# os.mkdir(folder)
# 
# for f in itc2007_track2_files:
#     
#     sub_folder = os.path.join(folder, f.split('\\')[-1].split('.')[0])
#     os.mkdir(sub_folder)
#     
#     date = time.time()
#     
#     problem = parse_itc2007_post_enrolment(f)[0]
#     search = SolutionSearch(problem)
#     solver = DepthFirstSearchSolver(search)
#     res = solver.solve(max_operations=10000)
#     
#     experiment_details = ExperimentDetails(
#         name='depth_first_search with 10k max operations',
#         instance_name=f.split('\\')[-1].split('.')[0],
#         instance_file_name = f.split('\\')[-1].split('.')[0],
#         dataset_name='itc2007_track2',
#         date=date,
#         results_data=res,
#         experiment_data={
#             "solver": 'depth_first_search',
#             "max_operations": 10000
#         }
#     )
#     
#     pickle.dump(experiment_details, open(os.path.join(sub_folder, 'experiment_details.pkl'), 'wb'))
#     
#     if res['success']:
#         np.save(os.path.join(sub_folder, 'solution gene.npy'), search.get_result_as_gene())
#     
#     plot_row_over_operation(os.path.join(sub_folder, 'row_over_operation_graph.png'), res['operation_history'], 'Depth first search max 10k operations for itc2007_track2 - ' + experiment_details.instance_name)
#     
#     plot_row_over_time(os.path.join(sub_folder, 'row_over_time_graph.png'),res['operation_history'], 'Depth first search max 10k operations for itc2007_track2 - ' + experiment_details.instance_name)
#     

## Depth first search with 10k operations

In [ ]:
# folder = 'output/itc2007_track3_depth_first_search_test_'+time.strftime("%Y%m%d-%H%M%S")
# os.mkdir(folder)
# 
# for f in itc2007_track3_files:
#     
#     
#     
#     date = time.time()
#     
#     problem = parse_itc2007_curriculum_based(f)[0]
#     search = SolutionSearch(problem)
#     solver = DepthFirstSearchSolver(search)
#     res = solver.solve(max_operations=10000)
#     
#     sub_folder = os.path.join(folder, problem.name)
#     os.mkdir(sub_folder)
#     
#     experiment_details = ExperimentDetails(
#         name='depth_first_search with 10k max operations',
#         instance_name=problem.name,
#         instance_file_name = f.split('\\')[-1].split('.')[0],
#         dataset_name='itc2007_track3',
#         date=date,
#         results_data=res,
#         experiment_data={
#             "solver": 'depth_first_search',
#             "max_operations": 10000
#         }
#     )
#     
#     pickle.dump(experiment_details, open(os.path.join(sub_folder, 'experiment_details.pkl'), 'wb'))
#     
#     if res['success']:
#         np.save(os.path.join(sub_folder, 'solution gene.npy'), search.get_result_as_gene())
#     
#     plot_row_over_operation(os.path.join(sub_folder, 'row_over_operation_graph.png'), res['operation_history'], 'Depth first search max 10k operations for itc2007_track3 - ' + experiment_details.instance_name)
#     
#     plot_row_over_time(os.path.join(sub_folder, 'row_over_time_graph.png'),res['operation_history'], 'Depth first search max 10k operations for itc2007_track3 - ' + experiment_details.instance_name)
#     

In [ ]:
# folder = 'output/itc2019_depth_first_search_test_'+time.strftime("%Y%m%d-%H%M%S")
# os.mkdir(folder)
# 
# for f in itc2019_files[5:6]:
# #for f in ['D:\\Desktop\\Datasets\\test\\2019maxblock.xml']: 
#     date = time.time()
#     
#     problem = parse_xml(f)[0]
#     search = SolutionSearch(problem)
#     solver = DepthFirstSearchSolver(search)
# 
#     try:
#         res = solver.solve(max_operations=10000)
#     except Exception as e:
#         res = {'success': False, 'error': str(e)}
#     
#     sub_folder = os.path.join(folder, problem.name)
#     os.mkdir(sub_folder)
#     
#     experiment_details = ExperimentDetails(
#         name='depth_first_search with 10k max operations',
#         instance_name=problem.name,
#         instance_file_name = f.split('\\')[-1].split('.')[0],
#         dataset_name='itc2019',
#         date=date,
#         results_data=res,
#         experiment_data={
#             "solver": 'depth_first_search',
#             "max_operations": 10000
#         }
#     )
#     
#     pickle.dump(experiment_details, open(os.path.join(sub_folder, 'experiment_details.pkl'), 'wb'))
#     
#     if res['success']:
#         np.save(os.path.join(sub_folder, 'solution gene.npy'), search.get_result_as_gene())
#     
#     plot_row_over_operation(os.path.join(sub_folder, 'row_over_operation_graph.png'), res['operation_history'], 'Depth first search max 10k operations for itc2019 - ' + experiment_details.instance_name)
#     
#     plot_row_over_time(os.path.join(sub_folder, 'row_over_time_graph.png'),res['operation_history'], 'Depth first search max 10k operations for itc2019 - ' + experiment_details.instance_name)
#     

## AC4 followed by Depth first search with 10k operations

In [ ]:
# folder = 'output/itc2007_track2_depth_first_search_test_'+time.strftime("%Y%m%d-%H%M%S")
# os.mkdir(folder)
# 
# for f in itc2007_track2_files:
#     
#     sub_folder = os.path.join(folder, f.split('\\')[-1].split('.')[0])
#     os.mkdir(sub_folder)
#     
#     date = time.time()
#     
#     problem = parse_itc2007_post_enrolment(f)[0]
#     search = SolutionSearch(problem)
#     
#     options_open_before_ac4 = np.count_nonzero(search.decision_table == 0)
#     
#     ac4_start_time = time.time()    
#     ac4 = AC4(search)
#     ac4.apply()    
#     ac4_end_time = time.time()
#     
#     options_open_after_ac4 = np.count_nonzero(search.decision_table == 0)
#     
#     solver = DepthFirstSearchSolver(search)
#     res = solver.solve(max_operations=10000)
#     
#     res['ac4_results'] = {
#         'options_open_before_ac4': options_open_before_ac4,
#         'options_open_after_ac4': options_open_after_ac4,
#         'options_closed_by_ac4': options_open_before_ac4 - options_open_after_ac4,
#         'ac4_time': ac4_end_time - ac4_start_time
#     }
#     
#     experiment_details = ExperimentDetails(
#         name='AC4 and depth_first_search with 10k max operations',
#         instance_name=f.split('\\')[-1].split('.')[0],
#         instance_file_name = f.split('\\')[-1].split('.')[0],
#         dataset_name='itc2007_track2',
#         date=date,
#         results_data=res,
#         experiment_data={
#             "solver": 'AC4 + depth_first_search',
#             "max_operations": 10000
#         }
#     )
#     
#     pickle.dump(experiment_details, open(os.path.join(sub_folder, 'experiment_details.pkl'), 'wb'))
#     
#     if res['success']:
#         np.save(os.path.join(sub_folder, 'solution gene.npy'), search.get_result_as_gene())
#     
#     plot_row_over_operation(os.path.join(sub_folder, 'row_over_operation_graph.png'), res['operation_history'], 'Depth first search max 10k operations for itc2007_track2 - ' + experiment_details.instance_name)
#     
#     plot_row_over_time(os.path.join(sub_folder, 'row_over_time_graph.png'),res['operation_history'], 'Depth first search max 10k operations for itc2007_track2 - ' + experiment_details.instance_name)
#     

In [ ]:
# folder = 'output/itc2007_track3_depth_first_search_test_'+time.strftime("%Y%m%d-%H%M%S")
# os.mkdir(folder)
# 
# for f in itc2007_track3_files:
#     
#     date = time.time()
#     
#     problem = parse_itc2007_curriculum_based(f)[0]
#     search = SolutionSearch(problem)
#     
#     options_open_before_ac4 = np.count_nonzero(search.decision_table == 0)
#     
#     ac4_start_time = time.time()    
#     ac4 = AC4(search)
#     ac4.apply()    
#     ac4_end_time = time.time()
#     
#     options_open_after_ac4 = np.count_nonzero(search.decision_table == 0)
#     
#     solver = DepthFirstSearchSolver(search)
#     res = solver.solve(max_operations=10000)
#     
#     res['ac4_results'] = {
#         'options_open_before_ac4': options_open_before_ac4,
#         'options_open_after_ac4': options_open_after_ac4,
#         'options_closed_by_ac4': options_open_before_ac4 - options_open_after_ac4,
#         'ac4_time': ac4_end_time - ac4_start_time
#     }
#     res = solver.solve(max_operations=10000)
#     
#     sub_folder = os.path.join(folder, problem.name)
#     os.mkdir(sub_folder)
#     
#     experiment_details = ExperimentDetails(
#         name='AC4 and depth_first_search with 10k max operations',
#         instance_name=problem.name,
#         instance_file_name = f.split('\\')[-1].split('.')[0],
#         dataset_name='itc2007_track3',
#         date=date,
#         results_data=res,
#         experiment_data={
#             "solver": 'AC4 + depth_first_search',
#             "max_operations": 10000
#         }
#     )
#     
#     pickle.dump(experiment_details, open(os.path.join(sub_folder, 'experiment_details.pkl'), 'wb'))
#     
#     if res['success']:
#         np.save(os.path.join(sub_folder, 'solution gene.npy'), search.get_result_as_gene())
#     
#     plot_row_over_operation(os.path.join(sub_folder, 'row_over_operation_graph.png'), res['operation_history'], 'Depth first search max 10k operations for itc2007_track3 - ' + experiment_details.instance_name)
#     
#     plot_row_over_time(os.path.join(sub_folder, 'row_over_time_graph.png'),res['operation_history'], 'Depth first search max 10k operations for itc2007_track3 - ' + experiment_details.instance_name)
#     

In [ ]:
# folder = 'output/itc2019_depth_first_search_test_'+time.strftime("%Y%m%d-%H%M%S")
# os.mkdir(folder)
# 
# for f in itc2019_files[5:6]:
# #for f in ['D:\\Desktop\\Datasets\\test\\2019maxblock.xml']: 
#     date = time.time()
# 
#     problem = parse_xml(f)[0]
#     search = SolutionSearch(problem)
# 
#     options_open_before_ac4 = np.count_nonzero(search.decision_table == 0)
# 
#     ac4_start_time = time.time()    
#     ac4 = AC4(search)
#     ac4.apply()    
#     ac4_end_time = time.time()
# 
#     options_open_after_ac4 = np.count_nonzero(search.decision_table == 0)
# 
#     solver = DepthFirstSearchSolver(search)
# 
#     try:
#         res = solver.solve(max_operations=10000)
#     except Exception as e:
#         res = {'success': False, 'error': str(e)}
# 
#     res['ac4_results'] = {
#         'options_open_before_ac4': options_open_before_ac4,
#         'options_open_after_ac4': options_open_after_ac4,
#         'options_closed_by_ac4': options_open_before_ac4 - options_open_after_ac4,
#         'ac4_time': ac4_end_time - ac4_start_time
#     }
# 
#     sub_folder = os.path.join(folder, problem.name)
#     os.mkdir(sub_folder)
# 
#     experiment_details = ExperimentDetails(
#         name='AC4 and depth_first_search with 10k max operations',
#         instance_name=problem.name,
#         instance_file_name = f.split('\\')[-1].split('.')[0],
#         dataset_name='itc2019',
#         date=date,
#         results_data=res,
#         experiment_data={
#             "solver": 'AC4 + depth_first_search',
#             "max_operations": 10000
#         }
#     )
# 
#     pickle.dump(experiment_details, open(os.path.join(sub_folder, 'experiment_details.pkl'), 'wb'))
# 
#     if res['success']:
#         np.save(os.path.join(sub_folder, 'solution gene.npy'), search.get_result_as_gene())
# 
#     plot_row_over_operation(os.path.join(sub_folder, 'row_over_operation_graph.png'), res['operation_history'], 'Depth first search max 10k operations for itc2019 - ' + experiment_details.instance_name)
# 
#     plot_row_over_time(os.path.join(sub_folder, 'row_over_time_graph.png'),res['operation_history'], 'Depth first search max 10k operations for itc2019 - ' + experiment_details.instance_name)


In [ ]:
run_experiment('Depth_first_search with 10k max operations', itc2007_track2_files, 'itc2007_track2', ac4_params=None, depth_first_params={'max_operations': 10000})

In [ ]:
run_experiment('Depth_first_search with 10k max operations', itc2007_track3_files, 'itc2007_track3', ac4_params=None, depth_first_params={'max_operations': 10000})

In [ ]:
run_experiment('Depth_first_search with 10k max operations', itc2019_files, 'itc2019', ac4_params=None, depth_first_params={'max_operations': 10000})

In [ ]:
run_experiment('AC4 and depth_first_search with 10k max operations', itc2007_track2_files, 'itc2007_track2', ac4_params={}, depth_first_params={'max_operations': 10000})

In [ ]:
run_experiment('AC4 and depth_first_search with 10k max operations', itc2007_track3_files, 'itc2007_track3', ac4_params={}, depth_first_params={'max_operations': 10000})

In [ ]:
run_experiment('AC4 and depth_first_search with 10k max operations', itc2019_files, 'itc2019', ac4_params={}, depth_first_params={'max_operations': 10000})